<a href="https://colab.research.google.com/github/XGHouFTW/ISEF-2022/blob/main/Merging_Dates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import datetime
from google.colab import drive
drive.mount('/content/drive')
basepath = "/content/drive/MyDrive/ISEF/Datasets/Historical Prices/BTC-USD.csv"
dfbase = pd.read_csv(basepath)
dfbase = dfbase.drop("Adj Close", 1)
#dfbase = dfbase.iloc[0:548,:] #Training set from Jan 31 2020 to July 31 2021

scorelist = {
    "btc_news":"/content/drive/MyDrive/ISEF/Datasets/Google News/btc_news_scored.csv",
    "eth_news":"/content/drive/MyDrive/ISEF/Datasets/Google News/eth_news_scored.csv",
    "doge_news":"/content/drive/MyDrive/ISEF/Datasets/Google News/doge_news_scored.csv",
    "cryptocurrency_news":"/content/drive/MyDrive/ISEF/Datasets/Google News/cryptocurrency_news_scored.csv",
    "economy_news":"/content/drive/MyDrive/ISEF/Datasets/Google News/economy_news_scored.csv",
    "finance_news":"/content/drive/MyDrive/ISEF/Datasets/Google News/finance_news_scored.csv",
    "politics_news":"/content/drive/MyDrive/ISEF/Datasets/Google News/politics_news_scored.csv",
    "pandemic_news":"/content/drive/MyDrive/ISEF/Datasets/Google News/pandemic_news_scored.csv"
} #dictionary of scorecolumnname and path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys


In [2]:
transMonth = {"Jan":"01","Feb":"02","Mar":"03","Apr":"04","May":"05","Jun":"06","Jul":"07","Aug":"08","Sep":"09","Oct":"10","Nov":"11","Dec":"12"}

a = "2020-01-31	" #from historical prices
def trans(strdate):
    return datetime.datetime(int(strdate[0:4]), int(strdate[5:7]), int(strdate[8:10]))

In [3]:
start = trans("2020-01-31") #to be compared to new dates for indexing
for i in range(len(dfbase)):
  dfbase.loc[i,"Date"] = trans(dfbase.loc[i,"Date"])

In [4]:
for scoreset in scorelist:
  dfAdd = pd.read_csv(scorelist[scoreset])
  dfbase[scoreset + "PosScoresum"] = 0
  dfbase[scoreset + "PosScoreaverage"] = 0
  dfbase[scoreset + "NegScoresum"] = 0
  dfbase[scoreset + "NegScoreaverage"] = 0
  dfbase[scoreset + "ComScoresum"] = 0
  dfbase[scoreset + "ComScoreaverage"] = 0

  dfbase[scoreset + "Scorecount"] = 0

  for i in range(len(dfAdd)):
    ind = (trans(dfAdd.loc[i,"Date"])-start).days
    if ind < 0 or ind > 731:
      print("Date out of range error:" + scoreset + str(i) + dfAdd.loc[i,"Date"])
    else:
      dfbase.loc[ind, scoreset + "PosScoresum"] = dfbase.loc[ind, scoreset + "PosScoresum"] + dfAdd.loc[i,"Pos"] #depends on if we only use compound or others too
      dfbase.loc[ind, scoreset + "NegScoresum"] = dfbase.loc[ind, scoreset + "NegScoresum"] + dfAdd.loc[i,"Neg"] #depends on if we only use compound or others too
      dfbase.loc[ind, scoreset + "ComScoresum"] = dfbase.loc[ind, scoreset + "ComScoresum"] + dfAdd.loc[i,"Com"] #depends on if we only use compound or others too
      dfbase.loc[ind, scoreset + "Scorecount"] = dfbase.loc[ind, scoreset + "Scorecount"] + 1
  
  for i in range(len(dfbase)):
    if not dfbase.loc[i,scoreset + "Scorecount"] == 0:
      dfbase[scoreset + "PosScoreaverage"] = dfbase[scoreset + "PosScoresum"]/dfbase[scoreset + "Scorecount"]
      dfbase[scoreset + "NegScoreaverage"] = dfbase[scoreset + "NegScoresum"]/dfbase[scoreset + "Scorecount"]
      dfbase[scoreset + "ComScoreaverage"] = dfbase[scoreset + "ComScoresum"]/dfbase[scoreset + "Scorecount"]


Date out of range error:btc_news141342022-02-10
Date out of range error:eth_news13192022-02-05
Date out of range error:eth_news81772022-02-09
Date out of range error:eth_news93392022-02-10
Date out of range error:cryptocurrency_news102020-01-29
Date out of range error:economy_news153992022-02-21
Date out of range error:finance_news142452022-02-09
Date out of range error:finance_news160112022-02-10
Date out of range error:finance_news165642022-02-15
Date out of range error:finance_news176222022-02-17
Date out of range error:politics_news4682022-02-16
Date out of range error:politics_news74052022-02-10
Date out of range error:politics_news107462022-02-11
Date out of range error:politics_news158192022-02-17
Date out of range error:politics_news185802022-02-15
Date out of range error:politics_news235592022-02-02
Date out of range error:pandemic_news58872022-02-15
Date out of range error:pandemic_news116072022-02-02
Date out of range error:pandemic_news167472022-02-21
Date out of range erro

In [11]:
dfbase.to_csv("/content/drive/MyDrive/ISEF/Datasets/MergedPriceData.csv")